In [76]:
import pandas as pd
import numpy as np
import re

# Data Visualization and Conclusions

In [77]:
train_data = pd.read_csv("train-test-datasets/train.csv")
test_data = pd.read_csv("train-test-datasets/test.csv")

submission = pd.read_csv("train-test-datasets/sample_submission.csv")
train_data.sample(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
3334,3584_01,Earth,False,F/676/S,TRAPPIST-1e,55.0,False,42.0,4.0,1215.0,0.0,3.0,Gayla Leonzaley,True
4269,4544_01,Earth,True,G/743/P,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Regana Pottney,True
6573,6932_01,Earth,False,G/1130/S,TRAPPIST-1e,20.0,False,559.0,0.0,83.0,112.0,0.0,Shaele Ewiseston,True
6678,7041_01,Mars,True,F/1343/S,TRAPPIST-1e,18.0,False,0.0,0.0,0.0,0.0,0.0,Picers Panie,True
6642,7004_01,Earth,False,F/1337/S,55 Cancri e,14.0,False,NaN,1.0,400.0,297.0,0.0,Karena Woodwardy,False


In [78]:
# train_data = train_data.dropna()
# test_data = test_data.dropna()

# test_data.sample(5)

Splitting Cabin into three different columns

In [ ]:
def cabin_split(data):
    # Compile the expected format: "Deck/Number/Side" like "B/123/P"
    pattern = re.compile(r'^([a-zA-Z]+)/(\d+)(/[a-zA-Z]+)?$')

    def split_cabin(val):
        if pd.isna(val):
            return pd.Series([pd.NA, pd.NA, pd.NA])
        if not pattern.match(val):
            print(f"Invalid format: {val}")
            return pd.Series([pd.NA, pd.NA, pd.NA])
        parts = val.split("/")
        deck = parts[0] if len(parts) > 0 else pd.NA
        number = parts[1] if len(parts) > 1 else pd.NA
        side = parts[2] if len(parts) > 2 else pd.NA
        return pd.Series([deck, number, side])

    # Apply the split_cabin function to each row
    data[["Cabin_deck", "Cabin_num", "Cabin_side"]] = data["Cabin"].apply(split_cabin)

    return data

train_data = cabin_split(train_data)
test_data = cabin_split(test_data)

test_data.sample(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin_deck,Cabin_num,Cabin_side
724,1482_01,Mars,True,F/284/S,TRAPPIST-1e,22.0,False,0.0,0.0,0.0,0.0,0.0,Prish Bancy,F,284,S
1053,2222_01,Mars,True,F/447/P,TRAPPIST-1e,19.0,False,0.0,0.0,0.0,0.0,0.0,Ockies Burie,F,447,P
3733,8146_01,Earth,True,G/1316/P,55 Cancri e,33.0,False,0.0,0.0,0.0,0.0,0.0,Isa Castron,G,1316,P
2476,5382_01,Europa,True,C/211/S,55 Cancri e,34.0,False,0.0,NaN,0.0,0.0,0.0,Tabdhib Cralinal,C,211,S
2780,6151_01,Europa,False,NaN,TRAPPIST-1e,33.0,False,0.0,187.0,0.0,2758.0,564.0,Achium Hometerval,<NA>,<NA>,<NA>


One-hot encoding the column - Destination

In [ ]:
print(set(test_data["Destination"]))

# data = pd.get_dummies(data, columns=['Destination'], prefix='Destin', prefix_sep='_')

# data.rename({ 
#     "Dest_55 Cancri e": "Destin_55_Cancri_e",
#     "Dest_PSO J318.5-22": "Destin_PSO_J318.5-22",
# }, inplace=True)

dest_split = {
    'TRAPPIST-1e': 1,
    '55 Cancri e': 2,
    'PSO J318.5-22': 3
}

train_data["Destination"] = train_data["Destination"].map(dest_split)
test_data["Destination"] = test_data["Destination"].map(dest_split)

# Model Training

## Data Cleaning